![RDD key pair](media/07.spark_statestore_files.png)

---
# 06 - Lectura y escritura de ficheros
--------------

### Sistemas de ficheros soportados
-   Igual que Hadoop, Spark soporta diferentes filesystems: local, HDFS, Amazon S3

    -   En general, soporta cualquier fuente de datos que se pueda leer con Hadoop

-   También, acceso a bases de datos relacionales o noSQL

    -   MySQL, Postgres, etc. mediante JDBC
    -   Apache Hive, HBase, Cassandra o Elasticsearch

### Formatos de fichero soportados

-   Spark puede acceder a diferentes tipos de ficheros:

    -   Texto plano, CSV, ficheros sequence, JSON, *protocol buffers* y *object files*
        -   Soporta ficheros comprimidos
    -   Veremos el acceso a algunos tipos en esta clase, y dejaremos otros para más adelante 

In [1]:
!pip install pyspark

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# Create apache spark context
from pyspark import SparkContext
sc = SparkContext(master="local", appName="Mi app")

In [13]:
# Stop apache spark context
sc.stop()

### Ejemplos con ficheros de texto

En el directorio `data/libros` hay un conjunto de ficheros de texto comprimidos.

In [1]:
# Ficheros de entrada
!ls data/libros

pg14329.txt.gz	pg17013.txt.gz	pg2000.txt.gz	pg25807.txt.gz	pg7109.txt.gz
pg1619.txt.gz	pg17073.txt.gz	pg24536.txt.gz	pg32315.txt.gz	pg8870.txt.gz
pg16625.txt.gz	pg18005.txt.gz	pg25640.txt.gz	pg5201.txt.gz	pg9980.txt.gz


### Funciones de lectura y escritura con ficheros de texto


- `sc.textFile(nombrefichero/directorio)` Crea un RDD a partir las líneas de uno o varios ficheros de texto
    - Si se especifica un directorio, se leen todos los ficheros del mismo, creando una partición por fichero
    - Los ficheros pueden estar comprimidos, en diferentes formatos (gz, bz2,...)
    - Pueden especificarse comodines en los nombres de los ficheros
- `sc.wholeTextFiles(nombrefichero/directorio)` Lee ficheros y devuelve un RDD clave/valor
    - clave: path completo al fichero
    - valor: el texto completo del fichero
- `rdd.saveAsTextFile(directorio_salida)` Almacena el RDD en formato texto en el directorio indicado
    - Crea un fichero por partición del rdd

In [6]:
# Lee todos los ficheros del directorio
# y crea un RDD con las líneas
lineas = sc.textFile("data/libros")

# Se crea una partición por fichero de entrada
print("Número de particiones del RDD lineas = {0}".format(lineas.getNumPartitions()))

Número de particiones del RDD lineas = 15


In [7]:
# Obtén las palabras usando el método split (split usa un espacio como delimitador por defecto)
palabras = lineas.flatMap(lambda x: x.split())
print("Número de particiones del RDD palabras = {0}".format(palabras.getNumPartitions()))

Número de particiones del RDD palabras = 15


In [11]:
# Reparticiono el RDD en 4 particiones       
palabras2 = palabras.coalesce(4)
print("Número de particiones del RDD palabras2 = {0}".format(palabras2.getNumPartitions()))

Número de particiones del RDD palabras2 = 4


In [12]:
# Toma una muestra aleatoria de palabras
print(palabras2.takeSample(False, 10))

['en', 'labio', '_pescas_', 'ahumado', 'amena.', 'mucho', 'inocente.', 'sosona', 'Roma,', 'O']


In [15]:
# Lee los ficheros y devuelve un RDD clave/valor
# clave->nombre fichero, valor->fichero completo
prdd = sc.wholeTextFiles("data/libros/p*.gz")
print("Número de particiones del RDD prdd = {0}\n".format(prdd.getNumPartitions()))

Número de particiones del RDD prdd = 1



In [16]:
# Obtiene un lista clave/valor
# clave->nombre fichero, valor->numero de palabras
lista = prdd.mapValues(lambda x: len(x.split())).collect()

for libro in lista:
    print("El fichero {0:14s} tiene {1:6d} palabras".format(libro[0].split("/")[-1], libro[1]))

El fichero pg14329.txt.gz tiene 183777 palabras
El fichero pg1619.txt.gz  tiene 109878 palabras
El fichero pg16625.txt.gz tiene 170900 palabras
El fichero pg17013.txt.gz tiene 396086 palabras
El fichero pg17073.txt.gz tiene 309473 palabras
El fichero pg18005.txt.gz tiene  86446 palabras
El fichero pg2000.txt.gz  tiene 384258 palabras
El fichero pg24536.txt.gz tiene 134016 palabras
El fichero pg25640.txt.gz tiene 207338 palabras
El fichero pg25807.txt.gz tiene  15014 palabras
El fichero pg32315.txt.gz tiene  46142 palabras
El fichero pg5201.txt.gz  tiene  49441 palabras
El fichero pg7109.txt.gz  tiene  35037 palabras
El fichero pg8870.txt.gz  tiene  54348 palabras
El fichero pg9980.txt.gz  tiene  34014 palabras


## Ficheros Sequence
Ficheros clave/valor usados en Hadoop

-   Sus elementos implementan la interfaz [`Writable`](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/io/Writable.html)

In [18]:
rdd = sc.parallelize([("a",2), ("b",5), ("a",8)], 2)

# Salvamos el RDD clave valor como fichero de secuencias
rdd.saveAsSequenceFile("file:///tmp/sequenceoutdir2")

In [19]:
# Lo leemos en otro RDD
rdd2 = sc.sequenceFile("file:///tmp/sequenceoutdir2", 
                       "org.apache.hadoop.io.Text", 
                       "org.apache.hadoop.io.IntWritable")
                       
print("Contenido del RDD {0}".format(rdd2.collect()))

Contenido del RDD [('b', 5), ('a', 8), ('a', 2)]


## Formatos de entrada/salida de Hadoop
Spark puede interactuar con cualquier formato de fichero soportado por Hadoop 
- Soporta las APIs “vieja” y “nueva”
- Permite acceder a otros tipos de almacenamiento (no fichero), p.e. HBase o MongoDB, a través de `saveAsHadoopDataSet` y/o `saveAsNewAPIHadoopDataSet`


In [20]:
# Salvamos el RDD clave/valor como fichero de texto Hadoop (TextOutputFormat)
rdd.saveAsNewAPIHadoopFile("file:///tmp/hadoopfileoutdir", 
                            "org.apache.hadoop.mapreduce.lib.output.TextOutputFormat",
                            "org.apache.hadoop.io.Text",
                            "org.apache.hadoop.io.IntWritable")

In [23]:
!echo 'Directorio de salida'
!ls -l /tmp/hadoopfileoutdir
!cat /tmp/hadoopfileoutdir/part-r-00001

Directorio de salida
total 8
-rw-r--r-- 1 xergioalex xergioalex 0 Sep 23 18:28 _SUCCESS
-rw-r--r-- 1 xergioalex xergioalex 4 Sep 23 18:28 part-r-00000
-rw-r--r-- 1 xergioalex xergioalex 8 Sep 23 18:28 part-r-00001
b	5
a	8


In [24]:
# Lo leemos como fichero clave-valor Hadoop (KeyValueTextInputFormat)
rdd3 = sc.newAPIHadoopFile("file:///tmp/hadoopfileoutdir", 
                          "org.apache.hadoop.mapreduce.lib.input.KeyValueTextInputFormat",
                          "org.apache.hadoop.io.Text",
                          "org.apache.hadoop.io.IntWritable")
                          
print("Contenido del RDD {0}".format(rdd3.collect()))

Contenido del RDD [('a', '2'), ('b', '5'), ('a', '8')]
